In [1]:
import pandas as pd
import yfinance as yf

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('seaborn')

from add_indicators import bollinger_bands, rsi, macd, add_all_indicators

In [2]:
def show_all():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

def add_change(df):
    df['% Change'] = df['Close'].diff()
    df['% Change'] = df['% Change']/df['Close']
    df['% Change'] = df['% Change'].shift(periods=-1)
    
def add_volume(df):
    df['% Volume'] = df['Volume'].diff()
    df['% Volume'] = df['% Volume']/df['Volume']
    #df['% Volume'] = df['% Volume'].shift(periods=-1)

def preprocess(df):
    add_change(df)
    add_volume(df)
    add_all_indicators(df)
    return df.dropna()

def create_model(model, train_X, train_y):
    model.fit(train_X, train_y)
    return model

def run_predictions(name, model, val_X, val_y):
    model_predictions = model.predict(val_X)
    mae = mean_absolute_error(val_y, model_predictions)
    score = r2_score(val_y, model_predictions)
    return name, mae, score

def plot(start, end, df):
    plt.plot(df['% Change'].iloc[start:end], label='% Change')
    plt.plot(df['rfr'].iloc[start:end], label='rfr')
    plt.plot(df['xgb'].iloc[start:end], label='xgb')
    plt.title('Results')
    plt.legend()
    plt.show()
    
def show_results(results):
    for r in results:
        df[r[0]] = r[1].predict(df[features])
        print('Model: ', r[0])
        print('% MAE: ', r[2])
        print('Score: ', r[3])
        print()

In [6]:
ticker = yf.Ticker("spy")
df = ticker.history(period="1y", interval="1d")

df = preprocess(df)

#features = ['Close', 'bb_bbl', 'bb_bbh', 'rsi']
#features = ['bb_bbl', 'bb_bbh', 'rsi', 'macd', 'macd_diff', 'macd_signal']
#features = ['Close', 'bb_bbl', 'bb_bbh', 'rsi', 'macd', 'macd_diff', 'macd_signal', '% Volume']
'''
features = ['Close', '% Volume',
            'bb_bbh', 'bb_bbl', 'bb_avg', 'bb_bbh_ind', 'bb_bbl_ind',
            'bb_pband', 'bb_wband', 'rsi', 'macd', 'macd_diff', 'macd_signal',
            'don_h', 'don_l', 'don_m', 'don_p', 'don_w',
            'ema_9', 'sma_5', 'sma_10', 'sma_15', 'sma_30', 'sma_50']

features = ['Close',# '% Volume',
            'bb_bbh', 'bb_bbl', 'bb_avg',
            'macd', 'macd_signal',
            'don_h', 'don_l', 'don_m', 'don_p', 'don_w']
'''
features = ['Close', '% Volume', 'rsi',
            'bb_bbh', 'bb_bbl', 'bb_avg',
            'ema_9', 'sma_5', 'sma_10', 'sma_15', 'sma_30']

X = df[features].copy()
y = df['% Change'].copy()

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

In [15]:
%%time

parameters = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_depth': [8, 10, 12, 15],
    'gamma': [0.001, 0.005, 0.01, 0.02],
    'random_state': [42]
}

eval_set = [(train_X, train_y), (val_X, val_y)]

model = XGBRegressor(objective='reg:squarederror')
clf = GridSearchCV(model, parameters)
clf.fit(train_X, train_y)

print(f'Best params: {clf.best_params_}')
print(f'Best validation score = {clf.best_score_}')

model = XGBRegressor(**clf.best_params_, objective='reg:squarederror')
model.fit(train_X, train_y, eval_set=eval_set, verbose=False)

Best params: {'gamma': 0.005, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200, 'random_state': 42}
Best validation score = -0.06725465727808669
CPU times: user 1min 32s, sys: 1.33 s, total: 1min 33s
Wall time: 56.9 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.005, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=2, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
model_predictions = model.predict(val_X)
mae = mean_absolute_error(val_y, model_predictions)
score = r2_score(val_y, model_predictions)
print(mae, score)

0.007862546084482173 -0.06868601059031465


In [18]:
model = XGBRegressor(n_estimators=500, learning_rate=0.05, n_jobs=4)
model.fit(train_X, train_y, 
             early_stopping_rounds=5, 
             eval_set=[(val_X, val_y)],
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [19]:
model_predictions = model.predict(val_X)
mae = mean_absolute_error(val_y, model_predictions)
score = r2_score(val_y, model_predictions)
print(mae, score)

0.007706117881353226 -0.2282950051617363
